# Project 3: **Ask‑the‑Web Agent**

Welcome to Project 3! In this project, you will learn how to use tool‑calling LLMs, extend them with custom tools, and build a simplified *Perplexity‑style* agent that answers questions by searching the web.

## Learning Objectives  
* Understand why tool calling is useful and how LLMs can invoke external tools.
* Implement a minimal loop that parses the LLM's output and executes a Python function.
* See how *function schemas* (docstrings and type hints) let us scale to many tools.
* Use **LangChain** to get function‑calling capability
* Combine LLM with a web‑search tool to build a simple ask‑the‑web agent.
* Connect to external tools using **MCP (Model Context Protocol)**, a universal standard for LLM‑tool integration.
* Optionally build a UI using Chainlit to test your agent.

## Roadmap
0. Environment setup
1. Write simple tools and connect them to an LLM
2. Standardize tool calling with JSON schemas
3. Use LangGraph for tool calling
4. Build a Perplexity-style web-search agent
5. (Optional) MCP: connect to external tool servers
6. (Optional) A minimal UI

# 0- Environment setup

### Step 1: Create your environment and install dependencies 
Before we start coding, you need a reproducible setup. Open a terminal in the same directory as this notebook, and use Conda or uv to install the project dependencies.

#### Option 1: Conda


```bash
# Create and activate the conda environment
conda env create -f environment.yml && conda activate web_agent

```

#### Option 2: UV (faster)

If you prefer [uv](https://docs.astral.sh/uv/) over Conda:

```bash
# Install uv (skip if already installed)
curl -LsSf https://astral.sh/uv/install.sh | sh

# Create venv and install dependencies
uv venv .venv-web-agent-uv && source .venv-web-agent-uv/bin/activate
uv pip install -r requirements.txt
```

### Step 2: Register this environment as a Jupyter kernel
```bash
python -m ipykernel install --user --name=web_agent --display-name "web_agent"
```
Now open your notebook and switch to the `web_agent` kernel (Kernel → Change Kernel).

### Step 3: Set up Ollama

In this project, we use **Ollama** to load and use open-weight LLMs. We start with smaller models like `gemma3:1b` and then switch to larger models like `llama3.2:3b`.

Start the **Ollama** server in a terminal. This launches a local API endpoint that listens for LLM requests.

```bash
ollama serve
```

Downloads the model so you can run them locally without API calls. 
```bash
ollama pull gemma3:1b
ollama pull llama3.2:3b
```

You can explore other available models [here](https://ollama.com/library) and pull them to experiment with.

In [ ]:
# Quick check: is Ollama running?
# If this fails, open a terminal and run: ollama serve

import httpx

response = httpx.get("http://localhost:11434/api/tags", timeout=5)
models = [m["name"] for m in response.json().get("models", [])]
print(f"Ollama is running. Installed models: {models}")

## 1- Tool Calling

LLMs are strong at answering questions, but they cannot directly access external data such as live web results, APIs, or computations. In real applications, agents rarely rely only on their internal knowledge. They need to query APIs, retrieve data, or perform calculations to stay accurate and useful. Tool calling bridges this gap by allowing the LLM to request actions from the outside world.

<img src="assets/tools.png" width="700">

As show below, We first implement a tool, then describe the tool as part of the model's prompt. When the model decides that a tool is needed, it emits a structured output. A parser will detect this output, execute the corresponding function, and feed the result back to the LLM so the conversation continues.

<img src="assets/tool_flow.png" width="700">

In this section, you will implement a simple `get_current_weather` function and teach the `gemma3:1b` model to use it when required.

In [ ]:
# ---------------------------------------------------------
# Step 1: Implement the tool
# ---------------------------------------------------------
# You can either:
#   (a) Call a real weather API (for example, OpenWeatherMap), or
#   (b) Create a dummy function that returns a fixed response (e.g., "It is 23°C and sunny in San Francisco.")
#
# Output:
#   • Return a short, human-readable sentence describing the weather.
#
# Example expected behavior:
#   get_current_weather("San Francisco") → "It is 23°C and sunny in San Francisco."
#

"""
YOUR CODE HERE (~2-3 lines of code)
"""

In [ ]:
# ----------------------------------------------------------------------
# Step 2: Create a prompt to teach the LLM when and how to use your tool
# ----------------------------------------------------------------------
# What to include:
#   • A SYSTEM_PROMPT that tells the model about the tool use and describes the tool
#   • A USER_QUESTION with a user query that should trigger the tool.
#       Example: "What is the weather in San Diego today?"

"""
YOUR CODE HERE (~5-10 lines of code)
"""

Now that you have defined a tool and shown the model how to use it, the next step is to call the LLM using your prompt.

In [ ]:
# ---------------------------------------------------------
# Step 3: Call the LLM with your prompt
# ---------------------------------------------------------
# Task:
#   Send SYSTEM_PROMPT + USER_QUESTION to the model.
#
# Steps:
#   1. Create an Ollama client
#   2. Use chat.completions.create to send your prompt to gemma3:1b
#   3. Print the response.
#
# Expected:
#   The model should return something like:
#   TOOL_CALL: {"name": "get_current_weather", "args": {"city": "San Diego"}}
# ---------------------------------------------------------
from openai import OpenAI

"""
YOUR CODE HERE (~10 lines of code)
"""

In [ ]:
# ---------------------------------------------------------
# Step 4: Manually parse the LLM output and call the tool
# ---------------------------------------------------------
# Task:
#   Detect when the model requests a tool, extract its name and arguments,
#   and execute the corresponding function.
#
# Steps:
#   1. Search for the text pattern "TOOL_CALL:{...}" in the model output.
#   2. Parse the JSON inside it to get the tool name and args.
#   3. Call the matching function (e.g., get_current_weather).
#
# Expected:
#   You should see a line like:
#       Calling tool `get_current_weather` with args {'city': 'San Diego'}
#       Result: It is 23°C and sunny in San Diego.
# ---------------------------------------------------------

import re, json

"""
YOUR CODE HERE (~7-10 lines of code)
"""

# 2- Standardize tool calling

So far, we handled tool calling manually by writing a function, manually teaching the LLM about it, and write a regex to parse the output. This approach does not scale if we want to add more tools. Adding more tools would mean more `if/else` blocks and manual edits to the prompt.

To make the system flexible, we can standardize tool definitions by automatically reading each function's signature, converting it to a JSON schema, and passing that schema to the LLM. This way, the LLM can dynamically understand which tools exist and how to call them without requiring manual updates to prompts or conditional logic.

Next, you will implement a small helper that extracts metadata from functions and builds a schema for each tool.

In [ ]:
# ---------------------------------------------------------
# Generate a JSON schema for a tool automatically
# ---------------------------------------------------------
#
# Steps:
#   1. Rewrite the get_current_weather function with docstring and arg types
#   2. Use `inspect.signature` to automatically get function parameters and docstring
#   2. For each argument, record its name, type, and description.
#   3. Build a schema containing: name, description, and parameters.
#   4. Test your helper on `get_current_weather` and print the result.
#
# Expected:
#   A dictionary describing the tool (its name, args, and types).
# ---------------------------------------------------------

from pprint import pprint
import inspect

def get_current_weather(city: str, unit: str = "celsius") -> str:
    """
    YOUR CODE HERE (~2-3 lines of code)
    """

def to_schema(fn):
    sig = inspect.signature(fn)
    """
    YOUR CODE HERE (~8-10 lines of code)
    """

tool_schema = to_schema(get_current_weather)
pprint(tool_schema)

In [ ]:
# ---------------------------------------------------------
# Provide the tool schema to the model instead of prompt surgery
# ---------------------------------------------------------
# Goal:
#   Give the model a "menu" of available tools so it can choose
#   which one to call based on the user’s question.
#
# Steps:
#   1. Add an extra system message (e.g., name="tool_spec")
#      containing the JSON schema(s) of your tools.
#   2. Include SYSTEM_PROMPT and the user question as before.
#   3. Send the messages to the model (e.g., gemma3:1b).
#   4. Print the model output to see if it picks the right tool.
#
# Expected:
#   The model should produce a structured TOOL_CALL indicating
#   which tool to use and with what arguments.
# ---------------------------------------------------------

"""
YOUR CODE HERE (~5-10 lines of code)
"""

## 3- LangChain for Tool Calling

So far, you built a simple tool-calling pipeline. While this helps you understand the logic, it does not scale well when working with multiple tools, complex parsing, or multi-step reasoning. We have to write manual parsers, function calling logic, and adding responses back to the prompt.

LangChain simplifies this process. You only need to declare your tools, and its *Agent* abstraction handles when to call a tool, how to use it, and how to continue reasoning afterward. In this section, you will create a **ReAct** Agent (Reasoning + Acting). As shown below, the model alternates between reasoning steps and tool use wihtout any manual work.

<img src="assets/react.png" width="500">

The following links might be helpful for completing this section:
- [Create Agents](https://docs.langchain.com/oss/python/langchain/agents)
- [LangChain Tools](https://docs.langchain.com/oss/python/langchain/tools)
- [Ollama](https://docs.langchain.com/oss/python/integrations/chat/ollama)

In [ ]:
# ---------------------------------------------------------
# Step 1: Define tools for LangChain
# ---------------------------------------------------------
# Steps:
#   1. Keep your existing `get_current_weather` function as before.
#   2. Create a new function (e.g., get_weather) that calls it.
#   3. Add the `@tool` decorator so LangChain can register it automatically.
#
# Notes:
#   • The decorator converts your Python function into a standardized tool object.

from langchain_core.tools import tool

"""
YOUR CODE HERE (~5 lines of code)
"""

In [ ]:
# ---------------------------------------------------------
# Step 2: Create the Agent
# ---------------------------------------------------------
# Steps:
#   1. Create a gemma3:1b LLM instance 
#   2. Create the agent using create_agent
#   3. Test the agent with a natural question using agent.invoke

from langchain_ollama import ChatOllama
from langchain.agents import create_agent

"""
YOUR CODE HERE (~6 lines of code)
"""

### What just happened?
Your run failed because `gemma3:1b` does not support native tool calling (function calling). LangChain expects the model to return a structured tool-call object, but `gemma3:1b` can only return plain text, so the tool invocation step breaks.

### Why previosuly, our manual approach worked with any model?

In previous sections, we used **text-based tool calling**. We described the tool format in the system prompt. We asked the model to output `TOOL_CALL: {"name": ..., "args": ...}`. We then parsed this text with regex.

This works with **any model** (even small ones like `gemma3:1b`) because we're just asking the model to follow a certain structured output format.

### Why LangChain requires specific models?

LangChain relies on **native tool calling** and it expects a consistent structured output format irrespective of the model. Hence, it enfornces model outputs structured tool calls in a specific format. This requires models trained specifically for function calling

**Rule of thumb**: Models under 3B parameters typically lack native tool-calling capability.

| Model | Size | Native Tool Support | Notes |
|-------|------|---------------------|-------|
| `gemma3:1b` | 1B | No | Works for manual approach only |
| `llama3.2:1b` | 1B | No | Works for manual approach only |
| `llama3.2:3b` | 3B | Yes | Good balance of speed and capability |
| `gemma3` | 4B | Yes | Supports native tools |
| `mistral` | 7B | Yes | Strong tool support |

Let's fix the issue we observed in the previous cell.



In [ ]:
# ---------------------------------------------------------
# Step 2 (retry): Re-create the Agent with a native tool-calling LLM
# ---------------------------------------------------------
# Steps:
#   1. Create a llama3.2:3b LLM instance 
#   2. Create a system prompt to teach react-style reasoning to the LLM
#   3. Create the agent using create_agent
#   4. Test the agent with a natural question using agent.invoke

from langchain_ollama import ChatOllama
from langchain.agents import create_agent

"""
YOUR CODE HERE (~10 lines of code)
"""

## 4- Web Search Agent

Now that you know how to use LangChain with tools, let's build something useful. Instead of a toy get_weather tool, let create an agent that searches the web and answers questions using real results. In the next section, you will create a [DuckDuckGo](https://github.com/deedy5/ddgs) search tool and wire it into a ReAct agent.

In [ ]:
# ---------------------------------------------------------
# Step 1: Write a web search tool
# ---------------------------------------------------------
# Steps:
#   1. Write a helper function (e.g., search_web) that:
#        • Takes a query string
#        • Uses DuckDuckGo (DDGS) to fetch top results (titles + URLs)
#        • Returns them as a formatted string
#   2. Wrap it with the @tool decorator to make it available to LangChain.


from ddgs import DDGS
from langchain_core.tools import tool

"""
YOUR CODE HERE (~7 lines of code)
"""

In [ ]:
# ---------------------------------------------------------
# Step 2: Initialize the web-search agent
# ---------------------------------------------------------
# Steps:
#   1. Create an LLM (e.g., ChatOllama).
#   2. Add your `web_search` tool to the tools list.
#   3. Create the agent using create_agent.
#
# Expected:
#   The agent should be ready to accept user queries
#   and use your web search tool when needed.
# ---------------------------------------------------------

from langchain_ollama import ChatOllama
from langchain.agents import create_agent

"""
YOUR CODE HERE (~5 lines of code)
"""

In [ ]:
# ---------------------------------------------------------
# Step 3: Test your Ask-the-Web agent using agent.invoke
# ---------------------------------------------------------
"""
YOUR CODE HERE (~2-3 lines of code)
"""

## 5- (Optional) MCP: Model Context Protocol

Up to now, every tool you used started as a Python function you wrote and registered yourself. **MCP (Model Context Protocol)** lets you skip that step. Tools come from an external *server*, and your code just connects to it. Think of it like USB for AI tools: any MCP client can plug into any MCP server and immediately use whatever tools it offers.

Below, we connect to `mcp-server-fetch` (a ready-made server that can retrieve any URL) using the Python MCP SDK. We launch the server, discover its tools, and call one, all without writing a single `@tool` function. To learn more, read: https://github.com/modelcontextprotocol/servers/tree/main/src/fetch

> **LangChain integration:** The `langchain-mcp-adapters` package can convert MCP tools into LangChain-compatible tools automatically, so you can drop them straight into a ReAct agent like the ones in section 4.

In [ ]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
# Create an MCP client session and connect it to mcp-server-fetch.
# Follow this link: https://github.com/modelcontextprotocol/servers/tree/main/src/fetch

"""
YOUR CODE HERE (~10 lines of code)
"""

In [ ]:
from langchain_mcp_adapters.tools import load_mcp_tools

from langchain.agents import create_agent
from langchain_ollama import ChatOllama

# Load the tool using load_mcp_tools
# create agent with llm and tools same as before
# Fetch the content of a website like http://python.org


"""
YOUR CODE HERE (~10 lines of code)
"""

## 6- (Optional) A Minimal UI

[Chainlit](https://chainlit.io/) is a Python library designed specifically for building LLM and agent UIs. It provides:
- Built-in streaming support
- Message history
- Step visualization (see tool calls as they happen)
- No frontend code required

If you are interested, follow Chainlit's documentation to implement a simple UI for your agent. The process typically involves:

1. You write a Python file named `chainlit_app.py` with the agent creation logic as well as UI handlers (e.g.,`@cl.on_message`)
2. Run the file in your terminal with `chainlit run app.py`
3. A web UI opens automatically at `http://localhost:8000`

In [ ]:
%%writefile chainlit_app.py
# ---------------------------------------------------------
# Chainlit Web Search Agent

import chainlit as cl
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langchain_core.messages import AIMessage, ToolMessage
from ddgs import DDGS


# ---------------------------------------------------------
# Define the web search tool
# ---------------------------------------------------------

"""
YOUR CODE HERE (~5 lines of code)
"""

# ---------------------------------------------------------
# Create the agent (once at startup)
# ---------------------------------------------------------

"""
YOUR CODE HERE (~3 lines of code)
"""

# ---------------------------------------------------------
# Chainlit message handler
# ---------------------------------------------------------
@cl.on_message
async def handle_message(message: cl.Message):
    """Handle user messages and stream agent responses."""
    
    """
    YOUR CODE HERE
    """


# ---------------------------------------------------------
# Welcome message
# ---------------------------------------------------------
@cl.on_chat_start
async def start(): 
    """
    YOUR CODE HERE (~5 lines of code)
    """

## 🎉 Congratulations!

You have built a **web-enabled agent** from scratch: manual tool calling → JSON schemas → LangChain ReAct → web search → MCP → UI.

Next steps:
* Try adding more tools, such as news or finance APIs.
* Experiment with multiple tools, different models, and measure accuracy vs. hallucination.
* Explore the [MCP server registry](https://github.com/modelcontextprotocol/servers) for ready-made tool servers.

👏 **Great job!** Take a moment to celebrate. The techniques you implemented here power many production agents and chatbots.